## Script shows comparing two models
### It is comparing two FLAN Models - XL and XXL
### Script can be extended for more than 2 models of same or different family

In [ ]:
!pip3 install langchain

In [ ]:
import json, boto3
from typing import Dict
from langchain import PromptTemplate, SagemakerEndpoint, LLMChain
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.model_laboratory import ModelLaboratory

In [ ]:
## Create Content Handler for input and output 
## It would change per model. Current handler is for FLAN T5 models
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps({'text_inputs': prompt, **model_kwargs})
        return input_str.encode('utf-8')
      
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["generated_texts"][0]
    
content_handler = ContentHandler()

In [ ]:
# Launch FLAN T5 XL and XXL models from SageMaker JumpStart and update the endpoints here
flanxlendpoint = "jumpstart-XXXXXXXX-text2text-flan-t5-xl"
flanxxlendpoint = "jumpstart-XXXXXXXX-text2text-flan-t5-xxl"
region = boto3.Session().region_name

### Case1: Comparision of models (XL and XXL) with the same parameters

In [ ]:
flanxlllm = SagemakerEndpoint(
    endpoint_name = flanxlendpoint, 
    region_name = region, 
    model_kwargs={"temperature": 0, "max_length": 100},
    content_handler = content_handler
)
flanxxlllm = SagemakerEndpoint(
    endpoint_name = flanxxlendpoint, 
    region_name = region, 
    model_kwargs={"temperature": 0, "max_length": 100},
    content_handler = content_handler
)

#### Case1 - Example1

In [ ]:
template = "{question}"
prompt = PromptTemplate(template=template,input_variables=["question"])

In [ ]:
lab = ModelLaboratory.from_llms([
                                 flanxlllm, 
                                 flanxxlllm
                                 ], prompt=prompt)

In [ ]:
lab.compare("Tell me the recipe to cook white pasta")

#### Case1 - Example2

In [ ]:
q  = """\
Answer based on Context: \n
Luxembourg is a fastest growing economy in the middle of europe.
The GDP growth of Luxembourg is 3%. 
Luxembourg is multi-culture and key to schengen agreement. \n
what is the GDP grwoth of Luxembourg?
"""
prompt = PromptTemplate(template=template,input_variables=["question"])

In [ ]:
lab = ModelLaboratory.from_llms([
                                 flanxlllm, 
                                 flanxxlllm
                                 ], prompt=prompt)

In [ ]:
lab.compare(q)

### Case2: Comparision of a single model (XXL) with the different parameters

In [ ]:
temp1 = 0
temp2 = 1
max_len1=100
max_len2=200

In [ ]:
flanxxlllm1 = SagemakerEndpoint(
    endpoint_name = flanxxlendpoint, 
    region_name = region, 
    model_kwargs={"temperature": temp1, "max_length": max_len1},
    content_handler = content_handler
)

flanxxlllm2 = SagemakerEndpoint(
    endpoint_name = flanxxlendpoint, 
    region_name = region, 
    model_kwargs={"temperature": temp2, "max_length": max_len2},
    content_handler = content_handler
)

In [ ]:
template = "{question}"
prompt = PromptTemplate(template=template,input_variables=["question"])

In [ ]:
lab = ModelLaboratory.from_llms([
                                 flanxxlllm1, 
                                 flanxxlllm2
                                 ], prompt=prompt)

In [ ]:
lab.compare("Tell me the recipe to cook white pasta")